In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
from datetime import datetime

In [2]:
# Configurando as opções do Chrome
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36")
chrome_options.add_argument("accept-language=pt-BR,pt;q=0.9")
chrome_options.add_argument("referer=https://www.epocacosmeticos.com.br/pesquisa?q=")
chrome_options.add_argument("accept=text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8")
chrome_options.add_argument("accept-encoding=gzip, deflate, br")
chrome_options.add_argument("connection=keep-alive")
chrome_options.add_argument("upgrade-insecure-requests=1")
chrome_options.add_argument("cache-control=max-age=0")
#chrome_options.add_experimental_option("prefs", {
#    "profile.managed_default_content_settings.javascript": 2
#})

service = Service()
driver = webdriver.Chrome(service=service, options=chrome_options)


In [3]:
url = "https://www.epocacosmeticos.com.br/pesquisa?q=protetor%20solar%20facial"

driver.get(url)
#time.sleep(10)

In [4]:
#link = driver.find_elements(By.XPATH, "//a[@data-content-item='true']")
link = driver.find_elements(By.XPATH, "//a[@data-content-item='true']")
prod_link_list = []
prod_audit = []

for lk in link:
    href_value = lk.get_attribute("href")
    prod_link_list.append(href_value)
    prod_audit.append(lk.text)

#prod_link_list
#prod_audit

In [5]:
prod_price = []
prod_descr = []
prod_marca = []
prod_detalhe = []
prod_comentario = []
prod_aval = []
prod_dataref = []
prod_link_ok = []

# Obter data, hora e minuto atual
data_hora_atual = datetime.now()

for url in prod_link_list:
    driver.get(url)
    time.sleep(4)

    price = driver.find_elements(By.XPATH, "//div[@data-content-installments='true']")
    for p in price:
        if p.text.strip():
            
            prod_link_ok.append(url)    

            prod_price.append(p.text.replace('ou R$ ','').replace(' no cartão',''))

            h1_elements = driver.find_elements(By.TAG_NAME, 'h1')
            h2_elements = driver.find_elements(By.TAG_NAME, 'h2')

            # Filtrar pelo que possui a classe específica
            for h1_element in h1_elements:
                if 'sr-only' in h1_element.get_attribute('class'):
                    h1_text = h1_element.text
                    prod_descr.append(h1_text)

            for h2_element in h2_elements:
                if 'sr-only' in h2_element.get_attribute('class'):
                    h2_text = h2_element.text
                    prod_marca.append(h2_text)

            try:
                detalhe = driver.find_element(By.XPATH, "//*[@id='__next']/main/section[1]/div[2]/div[1]/div[4]/div")
                texto_detalhe = detalhe.text
            except:
                try:
                    detalhe = driver.find_element(By.XPATH, "//*[@id='__next']/main/section[1]/div[2]/div[1]/div[5]/div/div/div[3]")
                    texto_detalhe = detalhe.text
                except:
                    texto_detalhe = "SEM DETALHE"
            
            prod_detalhe.append(texto_detalhe)

            try:
                avaliacao = driver.find_element(By.XPATH, "//*[@id='__next']/main/section[1]/div[2]/div[1]/div[9]/div/div/div[1]/section[1]/div[1]/div[1]/span[1]")
                prod_aval.append(avaliacao.text)
            except:
                try:
                    avaliacao = driver.find_element(By.XPATH, "//*[@id='__next']/main/section[1]/div[2]/div[1]/div[10]/div/div/div[1]/section[1]/div[1]/div[1]/span[1]")
                    prod_aval.append(avaliacao.text)
                except:
                    avaliacao = "0"
                    prod_aval.append(avaliacao)

            #botao_load = driver.find_element(By.XPATH, "//*[@id='__next']/main/section[1]/div[2]/div[1]/div[9]/div/div/button")
            #botao_load.click()

            try:
                comentario = driver.find_element(By.XPATH, "//*[@id='__next']/main/section[1]/div[2]/div[1]/div[9]/div/div/div[2]/section[2]")
                prod_comentario.append(comentario.text)
            except:
                try:
                    comentario = driver.find_element(By.XPATH, "//*[@id='__next']/main/section[1]/div[2]/div[1]/div[10]/div/div/div[2]/section[2]")
                    prod_comentario.append(comentario.text)
                except:
                    prod_comentario.append("SEM COMENTARIOS")

            prod_dataref.append(data_hora_atual)
    
driver.quit()


In [6]:
tab_resultado = {
    "prod_descricao" : prod_descr,
    "prod_marca" : prod_marca,
    "prod_URL" : prod_link_ok,
    "prod_price" : prod_price,
    "prod_detalhe" : prod_detalhe,
    "prod_comentario" : prod_comentario,
    "prod_avaliacao" : prod_aval,
    "prod_dataref" : prod_dataref
}

df = pd.DataFrame(tab_resultado)

In [7]:
print(len(prod_descr))
print(len(prod_marca))
print(len(prod_link_ok))
print(len(prod_price))
print(len(prod_detalhe))
print(len(prod_comentario))
print(len(prod_aval))

20
20
20
20
20
20
20


In [8]:
df.to_csv("C:\\Users\\Amend\\Downloads\\EPOCA_TEMP_22082024_1640.csv", sep=';', index=False, encoding='utf-8-sig')